# Chat With Your Data

## Solution: Chat With Your Data

# Install libraries

In [ ]:
pip install openai

In [ ]:
pip install python-dotenv

In [ ]:
pip install langchain

In [ ]:
pip install langchain-openai

In [ ]:
pip install pypdf

In [ ]:
pip install faiss-cpu

In [ ]:
pip install langchainhub

## Load OpenAI API Key

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY=os.environ['OPENAI_API_KEY']

## Load vector database from disk

In [2]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS


db = FAISS.load_local("../faiss_index", 
                      OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY, model="text-embedding-3-small"), 
                      allow_dangerous_deserialization=True)

## Configure retriever
### Use the similarity search capabilities of a vector store to facilitate retrieval

In [3]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 6})

## Configure LLM

In [4]:
from langchain_openai import ChatOpenAI

#initialize the LLM we'll use - OpenAI GPT 3.5 Turbo
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo-0125")

## Define prompt with conversation history

In [5]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

system_prompt = """Given the chat history and a recent user question \
generate a new standalone question \
that can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed or otherwise return it as is."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

retriever_with_history = create_history_aware_retriever(
    llm, retriever, prompt
)

## Perform question answering

In [6]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(retriever_with_history, question_answer_chain)

In [7]:
from langchain_core.messages import HumanMessage

chat_history = []

question = "What is Aetherfloris Ventus?"

ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})

chat_history.extend([HumanMessage(content=question), ai_msg_1["answer"]])

print(ai_msg_1["answer"])

Aetherfloris Ventus is a celestial plant with petals lighter than air, appearing to float freely, nurtured by the winds and clouds. It challenges our understanding of the natural world with its delicate beauty, nearly invisible stem, and spectral display. In alchemy, its essence is considered a treasure that bestows lightness upon those who partake, encouraging thoughts to soar and dreams to take flight.


In [8]:
second_question = "What does a single drop of it do?"

ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})
print(ai_msg_2["answer"])

A single drop of Aetherfloris Ventus can lift the spirits, freeing the mind from earthly burdens and encouraging thoughts to soar. It induces a sense of lightness in both body and mind, allowing dreams to take flight and inspiring a feeling of weightlessness. The true nature of Aetherfloris Ventus remains elusive even to dedicated seekers, representing the mysteries of the natural world waiting to be discovered.


In [9]:
third_question = "How does it compare to Noctis Umbraherba?"
ai_msg_3 = rag_chain.invoke({"input": third_question, "chat_history": chat_history})
print(ai_msg_3["answer"])

Noctis Umbraherba is a dark plant thriving in the absence of light, with leaves darker than night that absorb shadows and blooming only under darkness. In contrast, Aetherfloris Ventus is a celestial plant with petals lighter than air, appearing to float freely, nurtured by the winds and clouds. While Noctis Umbraherba is shrouded in mystery and linked to the secrets of the night, Aetherfloris Ventus challenges understanding with its ethereal beauty and alchemical significance.


In [10]:
fourth_question = "Do you think the Biological section of the Voynich Manuscript is important?"
ai_msg_4 = rag_chain.invoke({"input": fourth_question, "chat_history": chat_history})
print(ai_msg_4["answer"])

Yes, the Biological section of the Voynich Manuscript is significant as it offers a unique blend of fantastical biology and precise anatomical detail, providing insights into ancient medical knowledge and alchemical practices. The detailed anatomical diagrams of mythical beings within this section may have served as a practical guide for medicinal and alchemical purposes, showcasing the interplay between science, magic, and art in historical contexts. The annotations accompanying these diagrams hint at the potential medicinal and pharmacological applications of the depicted organs, adding to the section's complexity and cultural significance.
